In [1]:
# Imports:
import numpy as np
import glob
import matplotlib.pyplot as plt

import os

from sklearn.metrics import *
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
length = 277
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputClassifier

from os import path

from tensorflow import keras

from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D
#from keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
import eager_ops_fixed_ig

from keras.constraints import maxnorm

from tensorflow.keras.optimizers import Nadam


from tensorflow.keras.losses import sparse_categorical_crossentropy
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import time
from sklearn import svm
from sklearn.model_selection import cross_val_score

tf.config.run_functions_eagerly(True)

# Load the training and testing data:
train_values = np.empty(shape=[0, length])
test_values = np.empty(shape=[0, length])

train_beats = glob.glob('./train_patients.csv')
test_beats = glob.glob('./test_patients.csv')

for j in train_beats:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    train_values = np.append(train_values, csvrows, axis=0)

for j in test_beats:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    test_values = np.append(test_values, csvrows, axis=0)
    
print(train_values.shape)
print(test_values.shape)

# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]

print(X_train.shape)

## created by Fani
def showResults(test, pred, model_name):
    accuracy = accuracy_score(test, pred)
    precision= precision_score(test, pred, average='macro')
    recall = recall_score(test, pred, average = 'macro')
    f1score_macro = f1_score(test, pred, average='macro') 
    f1score_micro = f1_score(test, pred, average='micro') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("Recall : {}".format(recall))
    print("f1score macro : {}".format(f1score_macro))
    print("f1score micro : {}".format(f1score_micro))
    cm=confusion_matrix(test, pred, labels=[1,2,3,4,5,6,7,8])
    return (model_name, round(accuracy,3), round(precision,3) , round(recall,3) , round(f1score_macro,3), 
            round(f1score_micro, 3), cm)

#### TRAINING STEP
@tf.function
def train_step(inputs, labels, model, lamb):
    '''
    Takes a single step of training. 

    Args:
        inputs: A tensor. A batch of input to the model.
        labels: A tensor. The labels to use when training.
        model: A tf.keras.Model object, or subclass thereof.
    Returns:
        The predictions of the model on the inputs. Useful if you need to update metrics after training.
    '''
    with tf.GradientTape() as tape:
        tape.watch(inputs)
        predictions = model(inputs, training=True)
    
        pred_loss = model.compiled_loss(labels, predictions)
        total_loss = pred_loss

        if len(model.losses) > 0:
            regularization_loss = tf.math.add_n(model.losses)
            total_loss = total_loss + regularization_loss

        attributions        = eager_ops_fixed_ig.expected_gradients(inputs, labels, model)

        standardised_attributions = (attributions - tf.math.reduce_mean(attributions)) / tf.math.reduce_std(attributions)

        attr_adj_differences = tf.experimental.numpy.diff(standardised_attributions, axis=1)

        total_variation_attributions = tf.reduce_sum(tf.math.abs(attr_adj_differences))

        attribution_loss = lamb * total_variation_attributions
        total_loss          = total_loss + attribution_loss

    gradients = tape.gradient(total_loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    #model.compiled_metrics.update_state(labels, predictions)
    return predictions




overall_start_time = time.time()

num_epochs = 10
batch_size = 128

X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)

X_train1 = X_train1.astype('float32')
X_test1 = X_test1.astype('float32')
y_train1 = to_categorical(y_train)
y_test1 = to_categorical(y_test)


overall_start_time = time.time()

train_acc_fn = tf.keras.metrics.CategoricalAccuracy()
test_acc_fn  = tf.keras.metrics.CategoricalAccuracy()

#lamb = 0.00011
lamb = 0.000007
repeat_num=1
fold_no = 1

repeat_start_time = time.time()

  # Define the model architecture
model = Sequential()

model.add(Conv1D(filters=128, kernel_size=16,padding='same', activation='relu',input_shape=(275,1), kernel_constraint=maxnorm(5)))
model.add(BatchNormalization())

model.add(Conv1D(filters=32, kernel_size=16,padding='same', activation='relu', kernel_constraint=maxnorm(5)))
model.add(BatchNormalization())

model.add(Conv1D(filters=9, kernel_size=16,padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=4,padding='same'))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(5)))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(9, activation='softmax'))
model.compile(optimizer='Nadam', loss='categorical_crossentropy',metrics=['accuracy'])

# Generate a print
print('------------------------------------------------------------------------')
print(f'Training for fold {fold_no} ...')


# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((X_train1, y_train1))
train_dataset = train_dataset.batch(batch_size)

# train model over epochs
for epoch in range(num_epochs):
    # Fit data to model
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        predictions = train_step(x_batch_train, y_batch_train, model, lamb)

    print('epoch: ', epoch+1)

pred_logits = []

test_dataset = tf.data.Dataset.from_tensor_slices((X_test1, y_test1))
test_dataset = test_dataset.batch(batch_size)
for step, (x_batch_test, y_batch_test) in enumerate(test_dataset):
    test_logits = model(x_batch_test)
    pred_logits.append(test_logits)
    test_acc_fn(y_batch_test, test_logits)
test_acc = test_acc_fn.result().numpy()
print('Fold test accuracy: {:.4f}'.format(test_acc))

pred = np.concatenate(pred_logits)

fold_pred_fname = './eg_gradients_l1_cnn_v6_P_pred_' + str(repeat_num) + '_' + str(fold_no) + '.csv'
fold_test_fname = './eg_gradients_l1_cnn_v6_P_test_' + str(repeat_num) + '_' + str(fold_no) + '.csv'


with open(fold_pred_fname, "wb") as fin:
    np.savetxt(fin, pred, delimiter=",",fmt='%s')

with open(fold_test_fname, "wb") as fin:
    np.savetxt(fin, y_test1, delimiter=",",fmt='%s')

print('trial accuracy: ', train_acc_fn(y_test1, pred).numpy())
scores = model.evaluate(X_test1, y_test1, verbose=0)

print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')


print('Time taken for repeat: ', time.time() - repeat_start_time)


print('Time taken: ', time.time() - overall_start_time)

Loading  ./train_patients.csv
Loading  ./test_patients.csv
(206312, 277)
(14380, 277)
(206312, 275)
------------------------------------------------------------------------
Training for fold 1 ...
epoch:  1
epoch:  2
epoch:  3
epoch:  4
epoch:  5
epoch:  6
epoch:  7
epoch:  8
epoch:  9
epoch:  10
Fold test accuracy: 0.9896
trial accuracy:  0.9896384


C:\Users\TheAn\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:4526: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Score: loss of 0.07023150473833084; accuracy of 98.96383881568909%
Time taken for repeat:  5952.224355220795
Time taken:  5963.820904016495
